In [ ]:
#"When it comes to the intensity of gentrification across the country—at least over the first 13 years of the 21st century—the District tops the list".https://dcist.com/story/19/03/19/d-c-has-had-the-most-gentrifying-neighborhoods-in-the-country-study-finds/

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
from scipy import stats

In [5]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation


In [8]:
LTS = pd.read_csv('listings.csv', delimiter = ',', header = 0)
NBH = pd.read_csv('neighbourhoods.csv', delimiter = ',', header = 0)
REW = pd.read_csv('reviews.csv', delimiter = ',', header = 0)
CAL = pd.read_csv('calendar.csv.gz', delimiter = ',', header = 0, low_memory=False)
LTS_GZ = pd.read_csv('listings.csv.gz', delimiter = ',', header = 0, low_memory=False)
REW_GZ = pd.read_csv('reviews.csv.gz', delimiter = ',', header = 0, low_memory=False)
NBH_J= pd.read_json('neighbourhoods.geojson', lines=True)

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)

####Begin Data Cleaning

In [9]:
LTS.drop(columns=['neighbourhood_group','last_review'],inplace=True)
NBH.drop(columns=['neighbourhood_group'],inplace=True)

In [10]:
LTS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9148 entries, 0 to 9147
Data columns (total 14 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              9148 non-null   int64  
 1   name                            9146 non-null   object 
 2   host_id                         9148 non-null   int64  
 3   host_name                       9147 non-null   object 
 4   neighbourhood                   9148 non-null   object 
 5   latitude                        9148 non-null   float64
 6   longitude                       9148 non-null   float64
 7   room_type                       9148 non-null   object 
 8   price                           9148 non-null   int64  
 9   minimum_nights                  9148 non-null   int64  
 10  number_of_reviews               9148 non-null   int64  
 11  reviews_per_month               6988 non-null   float64
 12  calculated_host_listings_count  91

In [19]:
CAL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339128 entries, 0 to 3339127
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 178.3+ MB


In [46]:
#rp=CAL.loc[CAL['listing_id']== 153545, ['price']]


In [47]:
#rp['price'] = rp['price'].astype(float)

ValueError: could not convert string to float: '$49.00'

In [11]:
list(LTS_GZ.columns)

['id',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',


In [15]:
#30+ COLUMNS DROPPED FROM LTS_GZ, FINAL SHAPE = (9148rows, 73columns)
# LTS_GZ.drop(columns=['listing_url','scrape_id', 'last_scraped', 'experiences_offered', 'host_thumbnail_url', 'host_picture_url','host_verifications','neighbourhood_group_cleansed', 'calendar_last_scraped', 'license'],inplace=True)
# LTS_GZ.drop(columns=['thumbnail_url','medium_url','picture_url','host_url','host_has_profile_pic','host_identity_verified', 'square_feet', 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights','maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated', 'cancellation_policy'],inplace=True)
# LTS_GZ.drop(columns=['xl_picture_url','bed_type','guests_included','has_availability','availability_30', 'availability_60', 'availability_60', 'jurisdiction_names', 'requires_license'],inplace=True)

In [16]:
LTS_GZ

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3663,Classic Rowhouse: Porch+ART+Parking,"Bright, charming, artful home on bus line lead...",NOTE: We are offering our home for rent here o...,"Bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,Transportation: Free off-street parking behind...,Entire House,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former Peace Corps Volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,"Brightwood Park, Crestwood, Petworth",Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.025540,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$154.00,NaN,NaN,$0.00,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,3670,Beautiful Sun-Lit U Street 1BR/1BA,Convenient location perfect for business trave...,"Sunny, bright private room. Includes a queen s...",Convenient location perfect for business trave...,We are nicely situated on a quiet residential ...,There is one small dog living with us. He is h...,Super convenient public transportation.,Private bedroom and en-suite bathroom.,We live in the residesnce.,"No pets, parties or smoking please.",4630,Sheila,2008-11-25,"Washington, District of Columbia, United States",I run a start-up food company with my partner ...,NaN,NaN,0%,f,NaN,1.0,1.0,"Washington, DC, United States",U Street Corridor,"Howard University, Le Droit Park, Cardozo/Shaw",Washington,DC,20009,D.C.,"Washington, DC",US,United States,38.91842,-77.027500,f,Townhouse,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",""Pets live on this pr...",$75.00,$600.00,NaN,$500.00,$25.00,$0.00,2,30,0,0,79,0,2015-03-18,2018-07-25,87.0,9.0,9.0,9.0,9.0,10.0,9.0,f,f,f,f,1,0,1,0,1.25
2,3686,Vita's Hideaway,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,We love that our neighborhood is up and coming...,We love love love meeting people from around t...,We love that it only takes about 10 minutes to...,NaN,We allow guests to dictate the level of intera...,*no illegal activity *no smoking *no pets *no ...,4645,Vita,2008-11-26,"Washington D.C., District of Columbia, United ...","I am a literary scholar, teacher, poet, vegan ...",within a day,100%,100%,f,Anacostia,2.0,2.0,"Washington, DC, United States",Anacostia,Historic Anacostia,Washington,DC,20020,D.C.,"Washington, DC",US,United States,38.86314,-76.988360,f,House,Private room,1,1.0,1.0,1.0,"{Internet,Wifi,Kitchen,""Free street parking"",""...",$55.00,$430.00,$975.00,$0.00,$0.00,$0.00,2,365,90,365,72,2,2010-11-01,2020-0

In [17]:
#reset the overall index of the table
LTS_GZ.reset_index()

,index,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,0,3663,Classic Rowhouse: Porch+ART+Parking,"Bright, charming, artful home on bus line lead...",NOTE: We are offering our home for rent here o...,"Bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,Transportation: Free off-street parking behind...,Entire House,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former Peace Corps Volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,"Brightwood Park, Crestwood, Petworth",Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.025540,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",$154.00,NaN,NaN,$0.00,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,1,3670,Beautiful Sun-Lit U Street 1BR/1BA,Convenient location perfect for business trave...,"Sunny, bright private room. Includes a queen s...",Convenient location perfect for business trave...,We are nicely situated on a quiet residential ...,There is one small dog living with us. He is h...,Super convenient public transportation.,Private bedroom and en-suite bathroom.,We live in the residesnce.,"No pets, parties or smoking please.",4630,Sheila,2008-11-25,"Washington, District of Columbia, United States",I run a start-up food company with my partner ...,NaN,NaN,0%,f,NaN,1.0,1.0,"Washington, DC, United States",U Street Corridor,"Howard University, Le Droit Park, Cardozo/Shaw",Washington,DC,20009,D.C.,"Washington, DC",US,United States,38.91842,-77.027500,f,Townhouse,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",""Pets live on this pr...",$75.00,$600.00,NaN,$500.00,$25.00,$0.00,2,30,0,0,79,0,2015-03-18,2018-07-25,87.0,9.0,9.0,9.0,9.0,10.0,9.0,f,f,f,f,1,0,1,0,1.25
2,2,3686,Vita's Hideaway,IMPORTANT NOTES * Carefully read and be sure t...,IMPORTANT NOTES * Airbnb keeps accurate track ...,IMPORTANT NOTES * Carefully read and be sure t...,We love that our neighborhood is up and coming...,We love love love meeting people from around t...,We love that it only takes about 10 minutes to...,NaN,We allow guests to dictate the level of intera...,*no illegal activity *no smoking *no pets *no ...,4645,Vita,2008-11-26,"Washington D.C., District of Columbia, United ...","I am a literary scholar, teacher, poet, vegan ...",within a day,100%,100%,f,Anacostia,2.0,2.0,"Washington, DC, United States",Anacostia,Historic Anacostia,Washington,DC,20020,D.C.,"Washington, DC",US,United States,38.86314,-76.988360,f,House,Private room,1,1.0,1.0,1.0,"{Internet,Wifi,Kitchen,""Free street parking"",""...",$55.00,$430.00,$975.00,$0.00,$0.00,$0.00,2,365,90,365,72,2,2010-

In [18]:
LTS_GZ.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9148 entries, 0 to 9147
Data columns (total 73 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            9148 non-null   int64  
 1   name                                          9146 non-null   object 
 2   summary                                       8923 non-null   object 
 3   space                                         7005 non-null   object 
 4   description                                   9036 non-null   object 
 5   neighborhood_overview                         6284 non-null   object 
 6   notes                                         4209 non-null   object 
 7   transit                                       6381 non-null   object 
 8   access                                        5428 non-null   object 
 9   interaction                                   5880 non-null   o

In [19]:
#####Change the strings to lower case 

#LTS_GZ = LTS_GZ.apply(lambda x: x.str.lower() if x.dtype == "object" else x) 
#LTS_GZ['summary','space', 'description','name', 'notes', 'transit', 'access'] = LTS_GZ['summary','space', 'description','name', 'notes', 'transit', 'access'].astype('str') 
#LTS_GZ['summary','space', 'description','name', 'notes', 'transit', 'access'] = LTS_GZ.str.lower(['summary','space', 'description','name', 'notes', 'transit', 'access'])
LTS_GZ.summary = LTS_GZ.summary.str.lower()
LTS_GZ.space = LTS_GZ.space.str.lower()
LTS_GZ.description = LTS_GZ.description.str.lower()
LTS_GZ.name = LTS_GZ.name.str.lower()
LTS_GZ.notes = LTS_GZ.notes.str.lower()
LTS_GZ.transit = LTS_GZ.transit.str.lower()
LTS_GZ.access= LTS_GZ.access.str.lower()
LTS_GZ.host_about= LTS_GZ.host_about.str.lower()

In [20]:
# #### First attempt at removing the "$" from money columns 

# LTS_GZ['price'] = LTS_GZ['price'].replace({'\$':''}, regex = True) 

# # # comma in the columns prevents the above command from being used in conjunction with astype(float)

# LTS_GZ['price'] = LTS_GZ['price'].replace({',':''}, regex = True).astype(float)

In [21]:
####Code below changes all objects in LTS_GZ to floats
change_money = ['weekly_price', 'monthly_price', 'security_deposit']
LTS_GZ[change_money] = LTS_GZ[change_money].replace({'\$':''}, regex = True)
LTS_GZ[change_money] = LTS_GZ[change_money].replace({',':''}, regex = True).astype(float)

In [22]:
#### CHECK that money values are now floats
LTS_GZ.loc[:,['weekly_price','monthly_price','security_deposit','price']].dtypes

weekly_price        float64
monthly_price       float64
security_deposit    float64
price               float64
dtype: object

In [20]:
LTS_GZ

,id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3663,classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,"Brightwood Park, Crestwood, Petworth",Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.025540,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,3670,beautiful sun-lit u street 1br/1ba,convenient location perfect for business trave...,"sunny, bright private room. includes a queen s...",convenient location perfect for business trave...,We are nicely situated on a quiet residential ...,there is one small dog living with us. he is h...,super convenient public transportation.,private bedroom and en-suite bathroom.,We live in the residesnce.,"No pets, parties or smoking please.",4630,Sheila,2008-11-25,"Washington, District of Columbia, United States",i run a start-up food company with my partner ...,NaN,NaN,0%,f,NaN,1.0,1.0,"Washington, DC, United States",U Street Corridor,"Howard University, Le Droit Park, Cardozo/Shaw",Washington,DC,20009,D.C.,"Washington, DC",US,United States,38.91842,-77.027500,f,Townhouse,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",""Pets live on this pr...",75.0,600.0,NaN,500.0,$25.00,$0.00,2,30,0,0,79,0,2015-03-18,2018-07-25,87.0,9.0,9.0,9.0,9.0,10.0,9.0,f,f,f,f,1,0,1,0,1.25
2,3686,vita's hideaway,important notes * carefully read and be sure t...,important notes * airbnb keeps accurate track ...,important notes * carefully read and be sure t...,We love that our neighborhood is up and coming...,we love love love meeting people from around t...,we love that it only takes about 10 minutes to...,NaN,We allow guests to dictate the level of intera...,*no illegal activity *no smoking *no pets *no ...,4645,Vita,2008-11-26,"Washington D.C., District of Columbia, United ...","i am a literary scholar, teacher, poet, vegan ...",within a day,100%,100%,f,Anacostia,2.0,2.0,"Washington, DC, United States",Anacostia,Historic Anacostia,Washington,DC,20020,D.C.,"Washington, DC",US,United States,38.86314,-76.988360,f,House,Private room,1,1.0,1.0,1.0,"{Internet,Wifi,Kitchen,""Free street parking"",""...",55.0,430.0,975.0,0.0,$0.00,$0.00,2,365,90,365,72,2,2010-11-01,2020-04-01,92.0,9.0,9.0,

In [23]:
#CHANGE 'id' to 'listing_id' to make joining with REW easier
LTS_GZ.rename(columns={'id': 'listing_id'}, inplace=True)
LTS.rename(columns={'id': 'listing_id'}, inplace=True)

In [24]:
REW

,listing_id,date
0,3663,2010-09-15
1,3663,2010-10-16
2,3663,2010-10-24
3,3663,2010-11-03
4,3663,2011-03-11
...,...,...
355262,43336077,2020-05-10
355263,43336077,2020-05-19
355264,43415299,2020-05-15
355265,43415299,2020-05-18


#GENTRIFIED NEIGHBORHODS ACCORDING TO https://ncrc.org/qualified-opportunity-zones-2018-gentrified-neighborhoods-2000-2017/ 
#and
#https://dcist.com/story/20/06/18/dc-is-the-13th-most-gentrified-us-city-study/
#Columbia Heights, Petworth, NoMa, the eastern and northern edges of Capitol Hill, Navy Yard, Anacostia, and Marshall Heights, Mount Pleasant, Brookland

In [26]:
#GENT_NBH is NEW dataframe for only gentrified neighborhoods

GENT_NBH = NBH.copy()
GENT_NBH = GENT_NBH.loc[[ 0,1,2,3,5,7,19,25,28], :]


In [62]:
GENT_NBH

,neighbourhood
0,"Brightwood Park, Crestwood, Petworth"
1,"Brookland, Brentwood, Langdon"
2,"Capitol Hill, Lincoln Park"
3,"Capitol View, Marshall Heights, Benning Heights"
5,"Cleveland Park, Woodley Park, Massachusetts Av..."
7,"Columbia Heights, Mt. Pleasant, Pleasant Plain..."
19,Historic Anacostia
25,"Near Southeast, Navy Yard"
28,"River Terrace, Benning, Greenway, Dupont Park"


In [31]:
GENT_REW = REW.copy()
GENT_REW.dtypes

listing_id     int64
date          object
dtype: object

In [32]:
GENT_REW['date'] = pd.to_datetime(GENT_REW['date'])

In [37]:
#CREATED NEW REVIEWS DF FROM 2010 TO 2013
dmask = (GENT_REW['date'] >= '2009-01-20') & (GENT_REW['date'] <= '2013-12-31')
dmask
GENT_REW = GENT_REW.loc[dmask]
GENT_REW

,listing_id,date
0,3663,2010-09-15
1,3663,2010-10-16
2,3663,2010-10-24
3,3663,2010-11-03
4,3663,2011-03-11
...,...,...
29924,1897093,2013-12-28
29925,1897093,2013-12-30
30286,1936620,2013-12-01
30297,1936730,2013-12-18


In [34]:
#Changed to lowercase
REW_GZ.comments = REW_GZ.comments.str.lower()
REW_GZ

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3663,97856,2010-09-15,211347,Véga,we really enjoyed our stay at shawn's and rich...
1,3663,120132,2010-10-16,196456,Brett,shawn and richard were fantastic! truly! in ad...
2,3663,125719,2010-10-24,246755,Levi,shawn and richard are wonderful hosts - very a...
3,3663,132840,2010-11-03,32703,Ranen,"sean and richard,\r\n\r\nthank you, thank you,..."
4,3663,196726,2011-03-11,393528,Emily,shawn and richard are wonderful hosts! they ar...
...,...,...,...,...,...,...
355262,43336077,624450501,2020-05-10,251002885,Raood,i like the location
355263,43336077,625334918,2020-05-19,3886091,Edward,smooth and easy check in. great place!
355264,43415299,624840186,2020-05-15,89555071,Zac,all i can say about darius' place is that it i...
355265,43415299,625241568,2020-05-18,221553016,Felix,darius was an amazing host. he ensured that my...


In [38]:
GENT_REW_GZ = REW_GZ.copy()
GENT_REW_GZ['date'] = pd.to_datetime(GENT_REW_GZ['date'])

In [39]:
dmask2 = (GENT_REW_GZ['date'] >= '2009-01-20') & (GENT_REW_GZ['date'] <= '2013-12-31')

In [40]:
GENT_REW_GZ = GENT_REW_GZ.loc[dmask2]
GENT_REW_GZ

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,3663,97856,2010-09-15,211347,Véga,we really enjoyed our stay at shawn's and rich...
1,3663,120132,2010-10-16,196456,Brett,shawn and richard were fantastic! truly! in ad...
2,3663,125719,2010-10-24,246755,Levi,shawn and richard are wonderful hosts - very a...
3,3663,132840,2010-11-03,32703,Ranen,"sean and richard,\r\n\r\nthank you, thank you,..."
4,3663,196726,2011-03-11,393528,Emily,shawn and richard are wonderful hosts! they ar...
...,...,...,...,...,...,...
29924,1897093,9427821,2013-12-28,287502,Ken,this is a great place in a great location at a...
29925,1897093,9471169,2013-12-30,4873259,Annie,great place!
30286,1936620,9008667,2013-12-01,906480,Guy,it's definitely one of my best airbnb experien...
30297,1936730,9291457,2013-12-18,10632053,Kristine,justin is a very hospitable person. i would d...


In [51]:
LTS_NEW = LTS.copy()

In [42]:
GENT_NBH_right = pd.merge(LTS_NEW, GENT_NBH, on='neighbourhood', how='right')
#GENT_NBH_left = pd.merge(LTS_NEW, GENT_NBH, on='neighbourhood', how='left')
#GENT_NBH_outer = pd.merge(LTS_NEW, GENT_NBH, on='neighbourhood', how='outer')
#GENT_NBH_inner = pd.merge(LTS_NEW, GENT_NBH, on='neighbourhood', how='inner')


In [52]:
GENT_NBH_right

,listing_id,name,host_id,host_name,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,3663,Classic Rowhouse: Porch+ART+Parking,4617,Shawn & Richard,"Brightwood Park, Crestwood, Petworth",38.95888,-77.02554,Entire home/apt,154,3,41,0.35,2,117
1,13824,Natural Setting in DC,54317,Chris & Erin,"Brightwood Park, Crestwood, Petworth",38.95845,-77.03764,Entire home/apt,100,2,167,1.31,1,200
2,24712,"Modern Luxury, Jacuzzi Tub & Fireplace",100727,Capital Vacation,"Brightwood Park, Crestwood, Petworth",38.94592,-77.02331,Entire home/apt,309,4,115,1.09,4,355
3,26147,Beautiful Cozy Full Bed Room,5782,Val,"Brightwood Park, Crestwood, Petworth",38.95313,-77.03583,Private room,40,28,9,0.07,1,0
4,72454,Modern Charm,100727,Capital Vacation,"Brightwood Park, Crestwood, Petworth",38.94684,-77.02294,Entire home/apt,221,4,162,1.44,4,346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2755,41984139,"Roomy Studio Near Eateries + Metro, w/ Concierge",48005494,Zeus,"Near Southeast, Navy Yard",38.88043,-77.00863,Entire home/apt,89,30,0,NaN,266,248
2756,41984156,Upscale 1BR Near US Capitol Bldg w/ Gym + Pets OK,48005494,Zeus,"Near Southeast, Navy Yard",38.88046,-77.00785,Entire home/apt,113,30,0,NaN,266,268
2757,42403607,"Ideal Navy Yard 1BR w/ Gym, Pool, W/D, nr. Cap...",107434423,Blueground,"Near Southeast, Navy Yard",38.87845,-77.00681,Entire home/apt,98,30,0,NaN,151,203
2758,43065641,"Smart Navy Yard 1BR w/ Gym, Doorman, Pool, nea...",107434423,Blueground,"Near Southeast, Navy Yard",38.87985,-77.00815,Entire home/apt,90,30,0,NaN,151,348


In [63]:
LTS_GZ_NEW = LTS_GZ.copy()

In [64]:
LTS_GZ_NEW.head(2)

,listing_id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3663,classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,"Brightwood Park, Crestwood, Petworth",Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.02554,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,3670,beautiful sun-lit u street 1br/1ba,convenient location perfect for business trave...,"sunny, bright private room. includes a queen s...",convenient location perfect for business trave...,We are nicely situated on a quiet residential ...,there is one small dog living with us. he is h...,super convenient public transportation.,private bedroom and en-suite bathroom.,We live in the residesnce.,"No pets, parties or smoking please.",4630,Sheila,2008-11-25,"Washington, District of Columbia, United States",i run a start-up food company with my partner ...,NaN,NaN,0%,f,NaN,1.0,1.0,"Washington, DC, United States",U Street Corridor,"Howard University, Le Droit Park, Cardozo/Shaw",Washington,DC,20009,D.C.,"Washington, DC",US,United States,38.91842,-77.02750,f,Townhouse,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",""Pets live on this pr...",75.0,600.0,NaN,500.0,$25.00,$0.00,2,30,0,0,79,0,2015-03-18,2018-07-25,87.0,9.0,9.0,9.0,9.0,10.0,9.0,f,f,f,f,1,0,1,0,1.25


In [74]:
LTS_GZ_NEW.rename(columns={'neighbourhood':'spec_neighbourhood'}, inplace=True)

In [75]:
LTS_GZ_NEW.rename(columns={'neighbourhood_cleansed':'neighbourhood'}, inplace=True)

In [76]:
GENT_NBH_LEFT_GZ = pd.merge(GENT_NBH, LTS_GZ_NEW, on='neighbourhood', how='left')

In [77]:
GENT_NBH_LEFT_GZ

,neighbourhood,listing_id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,spec_neighbourhood,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,"Brightwood Park, Crestwood, Petworth",3663,classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.02554,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,"Brightwood Park, Crestwood, Petworth",13824,natural setting in dc,our newly renovated basement apartment is comp...,welcome to your charming home in the woods but...,our newly renovated basement apartment is comp...,16th Street Heights is a quiet residential are...,we have two young kids (ages 3 & 5) and they a...,parking is readily available on the street in ...,when you arrive come to the front door and we ...,We are a text away to provide help and informa...,A few things: - Please remove your shoes when ...,54317,Chris & Erin,2009-11-14,"Washington, District of Columbia, United States",we love our secret natural paradise in dc. ch...,within a few hours,88%,100%,t,16th Street Heights,1.0,1.0,"Washington, DC, United States",16th Street Heights,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95845,-77.03764,t,Guest suite,Entire home/apt,4,1.0,0.0,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",100.0,613.0,2037.0,NaN,$15.00,$25.00,2,60,0,200,167,13,2009-12-07,2020-02-08,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,f,f,1,1,0,0,1.31
2,"Brightwood Park, Crestwood, Petworth",24712,"modern luxury, jacuzzi tub & fireplace",the illinois avenue suites is a 3-level row ho...,NaN,the illinois avenue suites is a 3-level row ho...,The Petworth neighborhood is nestled in the NW...,NaN,our home has a walk score of 78 out of 100. th...,our guest have total access to our entire home.,Please call or email us and we will get back w...,1.Parking. There is free off-street parking i...,100727,Capital Vacation,2010-03-29,"Washington, District of Columbia, United States",hello everyone. thank you so much for checking...,within a few hours,90%,98%,f,Petworth,2.0,2.0,"Washington, DC, United States",Petworth,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.94592,-77.02331,f,Townhouse,Entire home/apt,16,3.5,4.0,4.0,"{TV,""Cable TV"",Internet,Wifi,"

In [73]:
LTS_GZ_NEW

,listing_id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,neighbourhood,neighbourhood_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3663,classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,"Brightwood Park, Crestwood, Petworth",Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.025540,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,3670,beautiful sun-lit u street 1br/1ba,convenient location perfect for business trave...,"sunny, bright private room. includes a queen s...",convenient location perfect for business trave...,We are nicely situated on a quiet residential ...,there is one small dog living with us. he is h...,super convenient public transportation.,private bedroom and en-suite bathroom.,We live in the residesnce.,"No pets, parties or smoking please.",4630,Sheila,2008-11-25,"Washington, District of Columbia, United States",i run a start-up food company with my partner ...,NaN,NaN,0%,f,NaN,1.0,1.0,"Washington, DC, United States",U Street Corridor,"Howard University, Le Droit Park, Cardozo/Shaw",Washington,DC,20009,D.C.,"Washington, DC",US,United States,38.91842,-77.027500,f,Townhouse,Private room,2,1.0,1.0,1.0,"{Wifi,""Air conditioning"",""Pets live on this pr...",75.0,600.0,NaN,500.0,$25.00,$0.00,2,30,0,0,79,0,2015-03-18,2018-07-25,87.0,9.0,9.0,9.0,9.0,10.0,9.0,f,f,f,f,1,0,1,0,1.25
2,3686,vita's hideaway,important notes * carefully read and be sure t...,important notes * airbnb keeps accurate track ...,important notes * carefully read and be sure t...,We love that our neighborhood is up and coming...,we love love love meeting people from around t...,we love that it only takes about 10 minutes to...,NaN,We allow guests to dictate the level of intera...,*no illegal activity *no smoking *no pets *no ...,4645,Vita,2008-11-26,"Washington D.C., District of Columbia, United ...","i am a literary scholar, teacher, poet, vegan ...",within a day,100%,100%,f,Anacostia,2.0,2.0,"Washington, DC, United States",Anacostia,Historic Anacostia,Washington,DC,20020,D.C.,"Washington, DC",US,United States,38.86314,-76.988360,f,House,Private room,1,1.0,1.0,1.0,"{Internet,Wifi,Kitchen,""Free street parking"",""...",55.0,430.0,975.0,0.0,$0.00,$0.00,2,365,90,365,72,2,2010-11-01,2020-04-01,92.0,

# LTS = Listings
# NBH = Neighborhoods
# REW = Reviews 
# CAL = Calendars
# LTS_GZ = Listings (MASTER) 
# REW_GZ = Reviews (MASTER) 
# NBH_J= MAP
# GENT_NBH = NEW dataframe for only gentrified neighborhoods
# GENT_REW_GZ = df for reviews from 2010 to 2013
# GENT_NBH_LEFT_GZ = Merge of Gentrified Neighborhoods and Listings(MASTER)
# FINAL = Merge of REVIEW from 2010-2013, Gentrified Neighborhood list, 
# desc_words_df = description words 
# rew_words_count = reviews/comments words
# nbo_words_df = Neigborhood overview words 

In [ ]:
GENT_REW_GZ.to_csv("GENT_REW_GZ.csv",header=True,index=False)
GENT_NBH_LEFT_GZ.to_csv("GENT_NBH_LEFT_GZ.csv",header=True,index=False)
GENT_NBH

In [79]:
FINAL = pd.merge(GENT_REW_GZ, GENT_NBH_LEFT_GZ, on='listing_id', how='right')

In [80]:
FINAL.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3999 entries, 0 to 3998
Data columns (total 78 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   listing_id                                    3999 non-null   int64         
 1   id                                            1311 non-null   float64       
 2   date                                          1311 non-null   datetime64[ns]
 3   reviewer_id                                   1311 non-null   float64       
 4   reviewer_name                                 1311 non-null   object        
 5   comments                                      1311 non-null   object        
 6   neighbourhood                                 3999 non-null   object        
 7   name                                          3999 non-null   object        
 8   summary                                       3659 non-null   object

#### Getting a clearer picture of prevelant words used in GENT_REW_GZ('comments') and in GENT_NBH_LEFT_GZ('description')

In [84]:
import nltk # natural language toolkit
nltk.download()
from nltk.corpus import stopwords

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


In [82]:
# # getting list of all words 2010 to 2013
reviews = GENT_REW_GZ['comments'].tolist()
reviews_words = []
for comments in reviews:
    comments = str(comments).split()
    for word in comments:
        reviews_words.append(word.lower())

In [83]:
print(reviews_words[:10])

['we', 'really', 'enjoyed', 'our', 'stay', 'at', "shawn's", 'and', "richard's", 'place.']


In [85]:
filtered_words = [word for word in reviews_words if word not in stopwords.words('english')]
print(filtered_words[:30])

['really', 'enjoyed', 'stay', "shawn's", "richard's", 'place.', 'house', 'well-located', 'calm', 'area', 'near', 'restaurants', 'pubs.', 'tidy,', 'warm', 'nicely', 'decorated.', 'make', 'everything', 'make', 'feel', 'confortable', 'home', 'easier', 'us', 'speak', 'french', 'fluently:', 'breakfast,', 'use']


In [130]:

from collections import Counter # to count words in our list

# # plotting top 25 words used by the host in naming their home
 
rew_words_count = Counter(filtered_words).most_common()
rew_words_count = rew_words_count[:50]
rew_words_count

[('great', 2607),
 ('stay', 2229),
 ('place', 1818),
 ('apartment', 1616),
 ('would', 1452),
 ('us', 1365),
 ('nice', 1041),
 ('recommend', 1020),
 ('metro', 1004),
 ('location', 991),
 ('really', 947),
 ('dc', 926),
 ('everything', 891),
 ('house', 889),
 ('room', 820),
 ('time', 791),
 ('clean', 790),
 ('also', 789),
 ('comfortable', 786),
 ('good', 755),
 ('-', 741),
 ('definitely', 730),
 ('close', 705),
 ('well', 701),
 ('walk', 695),
 ('easy', 669),
 ('wonderful', 657),
 ('staying', 646),
 ('dc.', 641),
 ('get', 628),
 ('home', 623),
 ('even', 603),
 ('made', 584),
 ('highly', 556),
 ('perfect', 555),
 ('enjoyed', 550),
 ('helpful', 523),
 ('like', 520),
 ('one', 490),
 ('could', 488),
 ('clean,', 487),
 ('neighborhood', 485),
 ('stayed', 468),
 ('first', 461),
 ('stay.', 458),
 ('around', 453),
 ('host', 449),
 ('washington', 439),
 ('walking', 432),
 ('bus', 426)]

In [131]:
rew_words_df = pd.DataFrame(rew_words_count)
rew_words_df.head(30)

,0,1
0,great,2607
1,stay,2229
2,place,1818
3,apartment,1616
4,would,1452
5,us,1365
6,nice,1041
7,recommend,1020
8,metro,1004
9,location,991


In [141]:
rew_words_df.rename(columns={0:"review words", 1:"review word count"}, inplace=True)

In [142]:
rew_words_df

,review words,review word count
0,great,2607
1,stay,2229
2,place,1818
3,apartment,1616
4,would,1452
5,us,1365
6,nice,1041
7,recommend,1020
8,metro,1004
9,location,991


In [113]:
descrip  = GENT_NBH_LEFT_GZ ['description'].tolist()
desc_words = []
for comments in descrip:
    comments = str(comments).split()
    for word in comments:
        desc_words.append(word.lower())

In [114]:
print (desc_words[:30])

['bright,', 'charming,', 'artful', 'home', 'on', 'bus', 'line', 'leading', 'directly', 'to', 'monuments', '&', 'museums.', 'gourmet', 'kitchen', 'and', 'rustic', 'farm', 'table.', 'cozy', 'porch', 'to', 'relax', 'on', 'after', 'busy', 'dc', 'day.', 'king,', 'queen']


In [116]:
desc_filtered_words = [word for word in desc_words if word not in stopwords.words('english')]


['bright,', 'charming,', 'artful', 'home', 'bus', 'line', 'leading', 'directly', 'monuments', '&', 'museums.', 'gourmet', 'kitchen', 'rustic', 'farm', 'table.', 'cozy', 'porch', 'relax', 'busy', 'dc', 'day.', 'king,', 'queen', 'beds.', 'free', 'off-street', 'parking.', 'visit!', 'note:']


AttributeError: 'NoneType' object has no attribute 'sort_values'

In [132]:
 
desc_words_count = Counter(desc_filtered_words).most_common()
desc_words_count = desc_words_count[:50]
desc_words_count

[('metro', 2113),
 ('apartment', 1884),
 ('private', 1680),
 ('room', 1676),
 ('walk', 1588),
 ('-', 1549),
 ('access', 1437),
 ('full', 1419),
 ('bedroom', 1418),
 ('living', 1397),
 ('home', 1383),
 ('one', 1326),
 ('capitol', 1316),
 ('house', 1297),
 ('located', 1236),
 ('street', 1214),
 ('dc', 1130),
 ('kitchen', 1119),
 ('space', 1068),
 ('neighborhood', 1053),
 ('2', 1052),
 ('guests', 1050),
 ('two', 1049),
 ('blocks', 1047),
 ('parking', 973),
 ('available', 966),
 ('great', 950),
 ('bed', 938),
 ('&', 932),
 ('also', 914),
 ('queen', 884),
 ('basement', 870),
 ('columbia', 837),
 ('large', 791),
 ('eastern', 758),
 ('bus', 755),
 ('place', 745),
 ('away', 742),
 ('close', 738),
 ('national', 736),
 ('bathroom', 727),
 ('restaurants', 713),
 ('kitchen,', 708),
 ('1', 701),
 ('block', 698),
 ('walking', 698),
 ('heights', 680),
 ('new', 665),
 ('stay', 664),
 ('hill', 650)]

In [133]:
desc_words_df = pd.DataFrame(desc_words_count)
desc_words_df.head(30)

,0,1
0,metro,2113
1,apartment,1884
2,private,1680
3,room,1676
4,walk,1588
5,-,1549
6,access,1437
7,full,1419
8,bedroom,1418
9,living,1397


In [134]:
 desc_words_df.rename(columns={0:"description words", 1:"description word count"}, inplace=True)

In [135]:
 desc_words_df

,description words,description word count
0,metro,2113
1,apartment,1884
2,private,1680
3,room,1676
4,walk,1588
5,-,1549
6,access,1437
7,full,1419
8,bedroom,1418
9,living,1397


In [127]:
nbo  = GENT_NBH_LEFT_GZ ['neighborhood_overview'].tolist()
nbo_words = []
for comments in nbo:
    comments = str(comments).split()
    for word in comments:
        nbo_words.append(word.lower())

In [128]:
print (nbo_words[:30])

['the', 'neighborhood:', 'we', 'are', 'in', 'the', 'north', 'petworth/brightwood', 'park', 'neighborhood.', 'very', 'quiet', 'street,', 'one', 'block', 'from', 'city', 'park,', 'community', 'gardens,', 'and', 'recreation', 'center/playground.', 'you', 'can', 'walk', '2', 'blocks', 'to', 'walmart']


In [129]:
nbo_filtered_words = [word for word in nbo_words if word not in stopwords.words('english')]


In [137]:
 
nbo_words_count = Counter(nbo_filtered_words).most_common()
nbo_words_count = nbo_words_count[:50]
nbo_words_count

[('neighborhood', 1144),
 ('walk', 821),
 ('nan', 812),
 ('restaurants', 730),
 ('capitol', 687),
 ('street', 669),
 ('metro', 625),
 ('blocks', 608),
 ('eastern', 500),
 ('market', 498),
 ('great', 487),
 ('park', 473),
 ('columbia', 453),
 ('dc', 443),
 ('also', 438),
 ('within', 436),
 ('-', 433),
 ('located', 432),
 ('walking', 432),
 ('one', 430),
 ('hill', 425),
 ('national', 414),
 ('historic', 414),
 ('away', 388),
 ('restaurants,', 387),
 ('heights', 360),
 ('minute', 360),
 ('block', 325),
 ('house', 317),
 ('close', 311),
 ('local', 310),
 ('many', 310),
 ('best', 282),
 ('short', 281),
 ('minutes', 277),
 ('bars', 277),
 ('neighborhood.', 275),
 ('away.', 272),
 ('quiet', 271),
 ('petworth', 270),
 ('area', 266),
 ('grocery', 265),
 ('coffee', 260),
 ('distance', 256),
 ('home', 249),
 ('residential', 234),
 ('washington', 233),
 ('&', 220),
 ('market,', 218),
 ('bus', 213)]

In [139]:
nbo_words_df = pd.DataFrame(desc_words_count)
nbo_words_df.head(50)

,0,1
0,metro,2113
1,apartment,1884
2,private,1680
3,room,1676
4,walk,1588
5,-,1549
6,access,1437
7,full,1419
8,bedroom,1418
9,living,1397


In [140]:
nbo_words_df.rename(columns={0:"nbo words", 1:"nbo word count"}, inplace=True)

In [81]:
FINAL.head(30)

,listing_id,id,date,reviewer_id,reviewer_name,comments,neighbourhood,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,spec_neighbourhood,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,3663,97856.0,2010-09-15,211347.0,Véga,we really enjoyed our stay at shawn's and rich...,"Brightwood Park, Crestwood, Petworth",classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.02554,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,3663,120132.0,2010-10-16,196456.0,Brett,shawn and richard were fantastic! truly! in ad...,"Brightwood Park, Crestwood, Petworth",classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.02554,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
2,3663,125719.0,2010-10-24,246755.0,Levi,shawn and richard are wonderful hosts - very a...,"Brightwood Park, Crestwood, Petworth",classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,Washin

In [175]:
GENT_NBH_LEFT_GZ

,neighbourhood,listing_id,name,summary,space,description,neighborhood_overview,notes,transit,access,interaction,house_rules,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,street,spec_neighbourhood,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,weekly_price,monthly_price,security_deposit,cleaning_fee,extra_people,minimum_nights,maximum_nights,availability_90,availability_365,number_of_reviews,number_of_reviews_ltm,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,is_business_travel_ready,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,"Brightwood Park, Crestwood, Petworth",3663,classic rowhouse: porch+art+parking,"bright, charming, artful home on bus line lead...",note: we are offering our home for rent here o...,"bright, charming, artful home on bus line lead...",The neighborhood: We are in the North Petworth...,NaN,transportation: free off-street parking behind...,entire house,"We will not be present, but available by phone...",No smoking inside No pets,4617,Shawn & Richard,2008-11-25,"Washington D.C., District of Columbia, United ...","43 year old, former peace corps volunteer who ...",NaN,NaN,NaN,f,Manor Park,0.0,0.0,"Washington, DC, United States",Manor Park,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95888,-77.02554,t,Townhouse,Entire home/apt,4,3.5,4.0,2.0,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",154.0,NaN,NaN,0.0,$50.00,$0.00,3,30,28,117,41,0,2010-09-15,2018-05-21,97.0,10.0,10.0,10.0,10.0,9.0,10.0,f,f,f,f,2,1,1,0,0.35
1,"Brightwood Park, Crestwood, Petworth",13824,natural setting in dc,our newly renovated basement apartment is comp...,welcome to your charming home in the woods but...,our newly renovated basement apartment is comp...,16th Street Heights is a quiet residential are...,we have two young kids (ages 3 & 5) and they a...,parking is readily available on the street in ...,when you arrive come to the front door and we ...,We are a text away to provide help and informa...,A few things: - Please remove your shoes when ...,54317,Chris & Erin,2009-11-14,"Washington, District of Columbia, United States",we love our secret natural paradise in dc. ch...,within a few hours,88%,100%,t,16th Street Heights,1.0,1.0,"Washington, DC, United States",16th Street Heights,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.95845,-77.03764,t,Guest suite,Entire home/apt,4,1.0,0.0,2.0,"{Internet,Wifi,""Air conditioning"",Kitchen,""Fre...",100.0,613.0,2037.0,NaN,$15.00,$25.00,2,60,0,200,167,13,2009-12-07,2020-02-08,98.0,10.0,10.0,10.0,10.0,10.0,10.0,f,f,f,f,1,1,0,0,1.31
2,"Brightwood Park, Crestwood, Petworth",24712,"modern luxury, jacuzzi tub & fireplace",the illinois avenue suites is a 3-level row ho...,NaN,the illinois avenue suites is a 3-level row ho...,The Petworth neighborhood is nestled in the NW...,NaN,our home has a walk score of 78 out of 100. th...,our guest have total access to our entire home.,Please call or email us and we will get back w...,1.Parking. There is free off-street parking i...,100727,Capital Vacation,2010-03-29,"Washington, District of Columbia, United States",hello everyone. thank you so much for checking...,within a few hours,90%,98%,f,Petworth,2.0,2.0,"Washington, DC, United States",Petworth,Washington,DC,20011,D.C.,"Washington, DC",US,United States,38.94592,-77.02331,f,Townhouse,Entire home/apt,16,3.5,4.0,4.0,"{TV,""Cable TV"",Internet,Wifi,"

In [176]:
GENT_NBH_LEFT_GZ.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2760 entries, 0 to 2759
Data columns (total 73 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   neighbourhood                                 2760 non-null   object 
 1   listing_id                                    2760 non-null   int64  
 2   name                                          2760 non-null   object 
 3   summary                                       2691 non-null   object 
 4   space                                         2146 non-null   object 
 5   description                                   2728 non-null   object 
 6   neighborhood_overview                         1948 non-null   object 
 7   notes                                         1317 non-null   object 
 8   transit                                       2034 non-null   object 
 9   access                                        1789 non-null   o